In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
df_posts = pd.read_csv('/home/sam/data/stormfront/posts.csv', dtype={
    'author': np.dtype(np.object),
    'author_join_date': np.dtype(np.object),
    'author_location': np.dtype(np.object),
    'author_posts': np.dtype(np.float64),
    'author_title': np.dtype(np.object),
    'content': np.dtype(np.object),
    'post_in_thread': np.dtype(np.int64),
    'thread_url': np.dtype(np.object),
    'post_date': np.dtype(np.object),
    'post_id': np.dtype(np.int64)
}, parse_dates=['author_join_date', 'post_date'])
df_authors = pd.read_csv('~/far-right/stormfront/authors.csv', parse_dates=['author_join_date'])
df_titles = pd.read_csv('/home/sam/far-right/stormfront/thread-titles.csv')

In [3]:
df_subposts = pd.read_csv('/home/sam/data/stormfront/subposts.csv')

In [4]:
subposts_and_posts = pd.merge(df_subposts, df_posts, how='left', on='post_id', copy=False)
subposts_posts_and_titles = pd.merge(subposts_and_posts, df_titles, how='left', on='thread_url', copy=False)

subposts_posts_and_titles.head()

,post_id,parent_id,parent_content,content_x,video_links,author,author_join_date,author_location,author_posts,author_title,content_y,post_in_thread,thread_url,post_date,title
0,4230580,NaN,NaN,\nhttp://www.reuters.com/article/oddly...2&pag...,[],salvete amici,2005-10-04,"Christian, Free-market Libertarian",3602.0,Forum Member,"<div id=""post_message_4230580"">&#13;\n\t\t\t&#...",1,https://www.stormfront.org/forum/t390760,2007-05-24 15:19:00,Cat-Sized African Rats Plague Florida
1,4230591,4230580.0,NaN,I suppose they wouldn't go down well with a d...,[],Royal Blue,2005-10-04,NaN,1813.0,Forum Member,"<div id=""post_message_4230591"">&#13;\n\t\t\t&#...",2,https://www.stormfront.org/forum/t390760,2007-05-24 15:27:00,Cat-Sized African Rats Plague Florida
2,4230635,4230580.0,NaN,The blacks or the rats? confused.gif \n,[],ArcticThunder,2006-05-04,U.S.A.,3940.0,Account Inactive,"<div id=""post_message_4230635"">&#13;\n\t\t\t&#...",3,https://www.stormfront.org/forum/t390760,2007-05-24 15:52:00,Cat-Sized African Rats Plague Florida
3,4230666,4230580.0,NaN,OMG I'll have nightmares for weeks now...YUCK...,[],BeautynBrains1488,2007-05-04,Texico Amexica Where Wetbacks Rule,654.0,Forum Member,"<div id=""post_message_4230666"">&#13;\n\t\t\t&#...",4,https://www.stormfront.org/forum/t390760,2007-05-24 16:13:00,Cat-Sized African Rats Plague Florida
4,4230668,NaN,NaN,\r\n\t\t\t\r\n\t\t\tAre you talking about actu...,[],LOKI-Nevada,2002-03-04,Fled to Utah,7697.0,Account Disabled by Request,"<div id=""post_message_4230668"">&#13;\n\t\t\t&#...",5,https://www.stormfront.org/forum/t390760,2007-05-24 16:16:00,Cat-Sized African Rats Plague Florida


In [ ]:
threads_groupby = subposts_posts_and_titles.fillna('').groupby('thread_url')
urls = threads_groupby.thread_url.first()
threads = threads_groupby.content_x.apply(lambda x: '\n'.join(x))

In [ ]:
import textacy
threads_corpus = textacy.corpus.Corpus('en', texts=list(threads), metadatas=({'url': x} for x in urls))

In [ ]:
terms_list = (doc.to_terms_list(ngrams=1, named_entities=True, as_strings=True) for doc in threads_corpus)
doc_term_matrix, id2term = textacy.vsm.doc_term_matrix(
        terms_list, weighting='tfidf', normalize=True, smooth_idf=True,
        min_df=2, max_df=0.95, max_n_terms=200000)

In [ ]:
term2id = {}
for id, term in id2term.items():
    term2id[term] = id

In [ ]:
import scipy.sparse as sp

In [ ]:
dfs = textacy.vsm.get_doc_freqs(doc_term_matrix)

In [ ]:
from collections import defaultdict

def text_to_term_matrix(text, verbose=True):
    """This method constructs a sparse matrix from a text, by calculating
    tf-idf based off existing document frequencies.
    """
    terms_list = textacy.doc.Doc(text, lang='en').to_terms_list(ngrams=1, named_entities=True, as_strings=True)
    term_counts = defaultdict(int)
    not_found = []
    N = 0
    for term in terms_list:
        try:
            term_id = term2id[term]
            term_counts[term_id] += 1
            N += 1
        except KeyError:
            not_found.append(term)
    
    tfidfs = {}
    l2_norm_denominator = 0.0
    for term_id, count in term_counts.items():
        tf = count / float(N)
        
        # smoothing as done in https://github.com/chartbeat-labs/textacy/blob/master/textacy/vsm.py#L144
        idf = np.log(1 / dfs[term_id]) + 1
        
        tfidfs[term_id] = tf * idf
        l2_norm_denominator += (tf * idf) ** 2
    
    l2_norm_denominator = np.sqrt(l2_norm_denominator)
    if verbose:
        print('Normalizing by L2 norm', l2_norm_denominator)
    for term_id, tfidf in tfidfs.items():
        tfidfs[term_id] = tfidf / l2_norm_denominator
    
    if verbose and not_found:
        print('Not found: ', ', '.join(not_found))
    
    # Building sparse matrix
    data = []
    rows = []
    columns = []
    for term_id, tfidf in tfidfs.items():
        data.append(tfidf)
        columns.append(term_id)
        rows.append(0)
    
    term_matrix = sp.coo_matrix((data, (rows, columns)), shape=[1, 200000])
    return term_matrix

text_to_term_matrix('hello trump')

In [68]:
text_to_term_matrix(author_texts[0])

Normalizing by L2 norm 0.244423761163
Not found:  1/16th comanche indian, ass*s, ass*s, -cut, -deport


<1x200000 sparse matrix of type '<class 'numpy.float64'>'
	with 287 stored elements in COOrdinate format>

In [ ]:
author_texts = subposts_posts_and_titles.fillna('').groupby('author').content_x.agg(lambda x: '\n'.join(x))

In [69]:
author_topics = {}

for author, text in author_texts.iteritems():
    author_topics[author] = model_nmf.transform(text_to_term_matrix(author_texts[0], verbose=False))

ValueError: Array with wrong shape passed to NMF (input H). Expected (20, 200000), but got (20, 307) 

In [ ]:
%debug

In [ ]:
author_topic_matrix = np.vstack(author_topics.values())

In [ ]:
author_topics_200 = {}

for author, text in author_texts.iteritems():
    author_topics_200[author] = model_nmf_200.transform(text_to_term_matrix(author_texts[0], verbose=False))
    
author_topic_matrix_200 = np.vstack(author_topics_200.values())

In [24]:
from sklearn.manifold import TSNE

In [ ]:
tsne_20 = TSNE(n_components=2, random_state=0)

In [ ]:
tsne_200 = TSNE(n_components=2, random_state=0)
author_topic_tsne_200 = tsne_200.fit_transform(author_topic_matrix_200)

In [47]:
tsne_200_topics = TSNE(n_components=2, random_state=0)
author_topic_tsne_200_topics = tsne_200_topics.fit_transform(author_topic_matrix_200.T)

In [ ]:
author_topic_tsne_20 = tsne_20.fit_transform(author_topic_matrix)

In [31]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [43]:
model_terms = []
for topic_idx, top_terms in model_nmf.top_topic_terms(id2term, top_n=5):
    model_terms.append('\n'.join(top_terms))

In [33]:
model_terms_200 = []
for topic_idx, top_terms in model_nmf_200.top_topic_terms(id2term, top_n=3):
    model_terms_200.append(', '.join(top_terms))

In [55]:
author_posts = [float(df_authors[df_authors.author == author].author_posts) for author in author_texts.keys()]

In [52]:
sorted(((terms, score) for terms, score in zip(model_terms, author_topics['Glacier'].tolist()[0])), key=lambda x: x[1], reverse=True)[:5]

KeyError: 'Glacier'

In [51]:
get_author_topics = lambda author: sorted(((terms, score) for terms, score in zip(model_terms_200, author_topics[author].tolist()[0])), key=lambda x: x[1], reverse=True)[:5]

topics_per_author = [get_author_topics(x) for x in author_texts.keys()]

KeyError: '**JT**'

# Plot of authors, as related by topic

### N=200 topics

In [66]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource

x = tsne_200.embedding_[:, 0]
y = tsne_200.embedding_[:, 1]
author_names = list(author_texts.keys())#model_terms

# Radius of each point corresponds to the number of documents attributed to that author.
scale = .00001
#topic_sizes = [1 for a in author_names]
radii = [x * scale for x in author_posts]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            author_names=author_names,
            author_posts=author_posts,
            #author_topics = topics_per_author,
            radii=radii,
        )
    )

# Add author names and sizes to mouse-over info.
hover = HoverTool(
        tooltips=[
        ("author", "@author_names"),
        ("posts", "@author_posts"),
        #("topics", "@author_topics")
        ]
    )

p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', radius='radii', source=source, fill_alpha=0.6, line_color=None)
show(p)

# Plot of topics (n=200), as related by author

In [50]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource

x = tsne_200_topics.embedding_[:, 0]
y = tsne_200_topics.embedding_[:, 1]

# Radius of each point corresponds to the number of documents attributed to that author.
#scale = 0.00001
#topic_sizes = [1 for a in author_names]
#radii = [x * scale for x in author_posts]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            topic_terms=model_terms_200
            
            #radii=radii,
        )
    )

# Add author names and sizes to mouse-over info.
hover = HoverTool(
        tooltips=[
        ("topics", "@topic_terms")
        ]
    )

p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', size=30, source=source, fill_alpha=0.6, line_color=None)
show(p)

In [ ]:
doc_topic_matrix_nmf_200.shape

In [ ]:
model_nmf = textacy.tm.TopicModel('nmf', n_topics=20)

model_nmf.fit(doc_term_matrix)
doc_topic_matrix_nmf = model_nmf.transform(doc_term_matrix)

model_nmf_200 = textacy.tm.TopicModel('nmf', n_topics=200)

model_nmf_200.fit(doc_term_matrix)
doc_topic_matrix_nmf_200 = model_nmf_200.transform(doc_term_matrix)

In [9]:
doc_topic_matrix_20 = model_nmf.get_doc_topic_matrix(doc_term_matrix)
doc_topic_matrix_200 = model_nmf_200.get_doc_topic_matrix(doc_term_matrix)

/home/sam/text-analytics/venv/local/lib/python3.5/site-packages/textacy/tm/topic_model.py:196: RuntimeWarning: invalid value encountered in true_divide
  return doc_topic_matrix / np.sum(doc_topic_matrix, axis=1, keepdims=True)


In [16]:
df_topics_20 = pd.DataFrame(doc_topic_matrix_20, columns=['topic_{}'.format(i) for i in range(20)])

In [26]:
df_topics_20['thread_url'] = urls.values

In [31]:
df_topics_200 = pd.DataFrame(doc_topic_matrix_200, columns=['topic_{}'.format(i) for i in range(200)])

In [32]:
df_topics_200['thread_url'] = urls.values

In [30]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:////home/sam/data/stormfront/stormfront.sqlite')

df_topics_20.to_sql('topics_20', engine)

In [33]:
df_topics_200.to_sql('topics_200', engine)

In [41]:
import pickle as pickle

with open('/home/sam/data/stormfront/topic-models/model_nmf_20.pickle', 'wb') as f:
    pickle.dump(model_nmf, f, pickle.HIGHEST_PROTOCOL)

In [83]:
with open('/home/sam/data/stormfront/topic-models/model_nmf_200.pickle', 'wb') as f:
    pickle.dump(model_nmf_200, f, pickle.HIGHEST_PROTOCOL)

# Loading Saved Data

In [17]:
import pickle as pickle

with open('/home/sam/data/stormfront/topic-models/model_nmf_20.pickle', 'rb') as f:
    model_nmf = pickle.load(f)

In [18]:
with open('/home/sam/data/stormfront/topic-models/model_nmf_200.pickle', 'rb') as f:
    model_nmf_200 = pickle.load(f)

In [16]:
import textacy

# 

# Graph Analytics

Once the topic model is built, we need to:

* Construct a dataframe that contains:
  - author
  - thread_url
  - columns for each 

In [10]:
import networkx as nx

B = nx.Graph()
B.add_nodes_from(df_authors.author, bipartite=0)
B.add_nodes_from(df_titles.thread_url, bipartite=1)

In [ ]:
B.add_edges_from(df_posts.groupby(['author', 'thread_url']).first().reset_index()[['author', 'thread_url']].values)

In [ ]:
from networkx.algorithms import bipartite

threads_by_users = bipartite.overlap_weighted_projected_graph(B, df_titles.thread_url, jaccard=True)